<a href="https://colab.research.google.com/github/shivammehta007/NLPResearch/blob/master/RNN_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch
import torch.nn as nn

In [0]:
text = ['It is such a beautiful weather today' , 'I feel like going for a walk', 'Trees and sky also looks very beautiful']

# Create a Vocabulary of Characters

vocab = set(' '.join(text))

In [0]:
int2char = dict(enumerate(vocab))
char2int = {char : index for index, char in int2char.items()}

In [37]:
print(int2char, char2int)

{0: 's', 1: 'y', 2: 'n', 3: 'v', 4: 'c', 5: 'w', 6: 'k', 7: 'g', 8: 'a', 9: 'l', 10: 'b', 11: 'o', 12: 'h', 13: ' ', 14: 'f', 15: 'I', 16: 'r', 17: 'i', 18: 'T', 19: 'u', 20: 't', 21: 'e', 22: 'd'} {'s': 0, 'y': 1, 'n': 2, 'v': 3, 'c': 4, 'w': 5, 'k': 6, 'g': 7, 'a': 8, 'l': 9, 'b': 10, 'o': 11, 'h': 12, ' ': 13, 'f': 14, 'I': 15, 'r': 16, 'i': 17, 'T': 18, 'u': 19, 't': 20, 'e': 21, 'd': 22}


In [0]:
# Padding all the text to that of maximum length
max_length = len(max(text, key=len))

In [0]:
for i in range(len(text)):
    while len(text[i]) < max_length:
        text[i] += ' '

As we will predict next character in sequence , we will deveide the data into 
* Input
* Ground Truth/ Target

In [0]:
input_sequence = [text[i][:-1] for i in range(len(text))]
target_sequence = [text[i][1:] for i in range(len(text))]

In [41]:
print('Input_Sentence : {} \nOutput_Sentence: {}'.format(input_sequence[2], target_sequence[2]))

Input_Sentence : Trees and sky also looks very beautifu 
Output_Sentence: rees and sky also looks very beautiful


Lets Convert all the Text to sequence of integers from the vocabulary, since computers understand numbers better than text anyday

In [0]:
for i in range(len(text)):
    input_sequence[i] = [char2int[x] for x in input_sequence[i]]
    target_sequence[i] = [char2int[x] for x in target_sequence[i]]

In [43]:
print('Input_Sentence : {} \nOutput_Sentence: {}'.format(input_sequence[2], target_sequence[2]))

Input_Sentence : [18, 16, 21, 21, 0, 13, 8, 2, 22, 13, 0, 6, 1, 13, 8, 9, 0, 11, 13, 9, 11, 11, 6, 0, 13, 3, 21, 16, 1, 13, 10, 21, 8, 19, 20, 17, 14, 19] 
Output_Sentence: [16, 21, 21, 0, 13, 8, 2, 22, 13, 0, 6, 1, 13, 8, 9, 0, 11, 13, 9, 11, 11, 6, 0, 13, 3, 21, 16, 1, 13, 10, 21, 8, 19, 20, 17, 14, 19, 9]


In [0]:
dict_size = len(char2int)
seq_len = len(input_sequence[0])
batch_size = len(text)

In [0]:
# Create One Hot representation of it on dict size
def one_hot_representation(sequence, dict_size, seq_len, batch_size):
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    for i in range(batch_size):
        for u in range(seq_len):
            features[i, u, sequence[i][u]] = 1
    return features

In [47]:
input_sequence = one_hot_representation(input_sequence, dict_size, seq_len, batch_size)
print(input_sequence.shape)
print(input_sequence[0,0])

(3, 38, 23)
(3, 38, 23)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


In [0]:
device = torch.device('cuda') if torch.cuda.is_available else torch.device('cpu')

In [0]:
#Putting the input and output to Tensors 
input_sequence = torch.from_numpy(input_sequence)
target_sequence = torch.Tensor(target_sequence)


In [0]:
class Model(nn.Module):
    def __init__(self,  input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.rnn = nn.RNN(iinput_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size) 
